In [6]:
# Testing Model parameters
from models.parameters import ModelParams

params = ModelParams()
params

ModelParams(z_min=0.0, z_max=100.0, L_shaft=100.0, T=1.0, g=9.80665, WaveVector=9001698.147821756, A_z0=100.0, A_v0=-20.0, B_z0=0.0, B_v0=40.0, h_bar=1.0545718176461565e-34, c=299792458.0, w0=2696919063030079.5)

In [ ]:
# Testing Simulation Settings
from models.settings import SimulationSettings

# Testing setting the settings:
settings = SimulationSettings(
    hours=5.0,
    frep=0.5,
    start_hr=5.0,
    sensor_dt=10.0,
    N_sensors=5,
    random_noise=0.005,
    sensor_resolution=0.05,
    sensor_accuracy=0.0
)

settings.validate()
settings

SimulationSettings(hours=5.0, frep=0.5, start_hr=5.0, sensor_dt=10.0, N_sensors=5, random_noise=0.005, sensor_resolution=0.05, sensor_accuracy=0.0)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from models.temperature import TemperatureField

field = TemperatureField()

t = np.linspace(0, 24, 1000)
#z_vals = [0, 25, 50, 75, 100]

plt.figure(figsize=(8, 5))

Top = field.temperature(t, 100)
Bot = field.temperature(t, 0)
Mid = field.temperature(t, 50)

Mid_check = (Top + Bot) / 2


for i in range(len(t)):
    print(f"t={t[i]:.2f} hr: Top={Top[i]:.2f} C, Mid={Mid[i]:.2f} C, Bot={Bot[i]:.2f} C, Mid_check={Mid_check[i]:.2f} C, Diff={Mid[i]-Mid_check[i]:.6f} C")


t=0.00 hr: Top=288.92 C, Mid=292.45 C, Bot=294.22 C, Mid_check=291.57 C, Diff=0.883883 C
t=0.02 hr: Top=288.91 C, Mid=292.44 C, Bot=294.22 C, Mid_check=291.56 C, Diff=0.878307 C
t=0.05 hr: Top=288.90 C, Mid=292.43 C, Bot=294.22 C, Mid_check=291.56 C, Diff=0.872696 C
t=0.07 hr: Top=288.88 C, Mid=292.42 C, Bot=294.23 C, Mid_check=291.55 C, Diff=0.867050 C
t=0.10 hr: Top=288.87 C, Mid=292.41 C, Bot=294.23 C, Mid_check=291.55 C, Diff=0.861369 C
t=0.12 hr: Top=288.86 C, Mid=292.40 C, Bot=294.23 C, Mid_check=291.55 C, Diff=0.855655 C
t=0.14 hr: Top=288.85 C, Mid=292.39 C, Bot=294.23 C, Mid_check=291.54 C, Diff=0.849907 C
t=0.17 hr: Top=288.84 C, Mid=292.38 C, Bot=294.24 C, Mid_check=291.54 C, Diff=0.844125 C
t=0.19 hr: Top=288.83 C, Mid=292.37 C, Bot=294.24 C, Mid_check=291.53 C, Diff=0.838310 C
t=0.22 hr: Top=288.81 C, Mid=292.36 C, Bot=294.24 C, Mid_check=291.53 C, Diff=0.832462 C
t=0.24 hr: Top=288.80 C, Mid=292.35 C, Bot=294.25 C, Mid_check=291.52 C, Diff=0.826581 C
t=0.26 hr: Top=288.79

<Figure size 800x500 with 0 Axes>